In [0]:
  from google.colab import drive

  drive.mount('/content/gdrive',force_remount=True)
  root_path = 'gdrive/My Drive/data1/'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import scipy.io
import pandas as pd
from sklearn.preprocessing import StandardScaler 


In [0]:

Z1=[]
P1=[]

for q in range(100,131):
  filname = root_path + str(q) + ".mat"
  #import
  mat = scipy.io.loadmat(filname)['Ftr']
  L1=list(mat)

  # LRFs
  for i in range(0,500):
    
    K=np.ndarray.tolist(np.asarray(L1[0][0][i][0]))
    Z1.append(K)
  # print(Z1)

  # Neighbours
  for i in range(0,500):
    
    K=np.ndarray.tolist(np.asarray(L1[0][1][i][0]))
    P1.append(K)
  # print(P1)

  
  
    
  

In [0]:

for q in range(131,160):
  filname = root_path + str(q) + ".mat"
  #import
  mat = scipy.io.loadmat(filname)['Ftr']
  L1=list(mat)

  # LRFs
  for i in range(0,500):
    
    K=np.ndarray.tolist(np.asarray(L1[0][0][i][0]))
    Z1.append(K)
  # print(Z1)

  # Neighbours
  for i in range(0,500):
    
    K=np.ndarray.tolist(np.asarray(L1[0][1][i][0]))
    P1.append(K)
  # print(P1)

  
  

In [0]:

P1=np.asarray(P1)
Z1=np.asarray(Z1)

print(len(P1))



30000


In [0]:

sess = tf.Session()
print(sess.run(main_loss(a,b)))

Tensor("MatrixInverse:0", shape=(3, 3), dtype=float64)
Tensor("MatMul:0", shape=(3, 3), dtype=float64)
Tensor("Trace:0", shape=(), dtype=float64)
0.0


In [0]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(P1,Z1,test_size=0.4,random_state=0)

x_train=np.float32(x_train)

x_test=np.float32(x_test)

y_train=np.float32(y_train)

y_test=np.float32(y_test)

print(y_train.shape)


(18000, 3, 3)


In [0]:
# You will learn how to build very deep convolutional networks, using Residual Networks (ResNets)
# In theory, very deep networks can represent very complex functions; but in practice, they are hard to train. Residual Networks, introduced by He et al., allow you to train much deeper networks than were previously practically feasible.

# Let's import packages
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Dropout ,Activation, ZeroPadding1D, BatchNormalization, Flatten, SeparableConv1D, AveragePooling1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

# Identity block

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = SeparableConv1D(filters = F1, kernel_size = 1, strides = 1, padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization( name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    
    # Second component of main path 
    X = SeparableConv1D(filters = F2, kernel_size=(f), strides = (1), padding='same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization( name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = SeparableConv1D(filters = F3, kernel_size=(1), strides = 1, padding="valid", name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    
    return X





Using TensorFlow backend.


In [0]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    '''conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    '''
        eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Convolution2D(nb_filter1, 1, 1, subsample=strides,
                      name=conv_name_base + '2a', bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Convolution2D(nb_filter2, kernel_size, kernel_size,
                      name=conv_name_base + '2b', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Convolution2D(nb_filter3, 1, 1, subsample=strides,
                             name=conv_name_base + '1', bias=False)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = merge([x, shortcut], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

In [0]:
# def ResNet50():
    """
    Implementation of the popular ResNet50 the following architecture:
    SeparableConv1D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes
    Returns:
    model -- a Model() instance in Keras
    """
X_input=Input(shape=(1499,3))

    
    # Zero-Padding
X = ZeroPadding1D((3))(X_input)
    
    # Stage 1
X = SeparableConv1D(16, (7), strides = (2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling1D((4), strides=(4))(X)

    # Stage 2
X = convolutional_block(X, f = 3, filters = [16, 16, 32], stage = 2, block='a', s=1)
X = identity_block(X, 3, [16, 16, 32], stage=2, block='b')
X = identity_block(X, 3, [16, 16, 32], stage=2, block='c')

    # Stage 3
X = convolutional_block(X, f=3, filters = [32,32,64], stage = 3, block='a', s=2)
X = identity_block(X, 3, filters = [32,32,64],stage=3, block='b')
X = identity_block(X, 3, filters = [32,32,64], stage=3, block='c')
X = identity_block(X, 3, filters = [32,32,64], stage =3, block='d')
X = AveragePooling1D((3), name='avg_pool0')(X)

    


    # Stage 4 
X = convolutional_block(X, f=3, filters = [64,64,128],stage=4, block='a', s=2)
X = identity_block(X, 3, filters = [64,64,128], stage=4, block='b')
X = identity_block(X, 3, filters = [64,64,128], stage=4, block='c')
X = identity_block(X, 3, filters= [64,64,128], stage=4, block='d')
X = identity_block(X, 3, filters=[64,64,128], stage=4, block='e')
X = identity_block(X, 3, filters=[64,64,128], stage=4, block='f')

    # Stage 5 
X = convolutional_block(X, f=3, filters=[128,128,256], stage=5,block='a', s=3)
X = identity_block(X, 3, filters=[128,128,256], stage=5, block='b')
X = identity_block(X,3, filters=[128,128,256], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling1D(...)(X)"
X = AveragePooling1D((2), name='avg_pool')(X)
    
#X=Dropout(0.38)(X)
    # output layer
   # X = Flatten()(X)
X = Dense(3, activation='linear', name='fc' + str(3), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
model = Model(inputs = X_input, outputs = X, name='ResNet50')




    # Define the input as a tensor with shape input_shape
   
   # return model

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
from keras.optimizers import adam
# Run the following code to build the model's graph. If your implementation is not correct you will know it by checking your accuracy when running model.fit(...) below.
# model = ResNet50()
opt = adam(lr=0.1,clipnorm=1.0)

model.compile(loss='cosine_proximity', optimizer= opt ,metrics=['accuracy'])
model.summary()
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(x_train, y_train,batch_size=64,epochs=100,verbose=1,validation_data=(x_test, y_test))
score = model.evaluate(x_train, y_train, verbose=0)
trainerr = model.evaluate(x_test, y_test, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])
print('Test loss:',trainerr[0])
print('Test accuracy:',trainerr[1])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1499, 3)      0                                            
__________________________________________________________________________________________________
zero_padding1d_1 (ZeroPadding1D (None, 1505, 3)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (SeparableConv1D)         (None, 750, 16)      85          zero_padding1d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 750, 16)      64          conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
 import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('losss')
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

